In [1]:
import matplotlib.pyplot as plt
import cobra
import cbmpy
from cobra.io import validate_sbml_model
import libsbml
import importlib
from xml.etree import ElementTree
import Model_correction as mc
import cplex

Hep_G2_dir = "../Stage/liver_models/Hep-G2/Hep-G2.xml-5a91f1955e8a9c8a5c5d2ff4a1737c21/"
iHep_dir = "../Stage/liver_models/iHepatocytes2322/"
#import cplex

No solver present, unable to create shortcuts


AttributeError: module 'collections' has no attribute 'MutableMapping'

## Hep_G2 model : Modèle de cellule cancéreuse du foie. 

[Article](./Hep-G2/Hep-G2.pdf)

### Model loading

#### Original model loading

In [3]:
# Hep_G2 : Cancer liver from 2015
# Built from HMR 2.0 using tINIT.

#Hep_G2_temp = cbmpy.CBRead.readSBML2FBA('./Hep-G2/Hep-G2.xml-5a91f1955e8a9c8a5c5d2ff4a1737c21/Hep-G2.xml')


#path= './Hep-G2/Hep-G2.xml-5a91f1955e8a9c8a5c5d2ff4a1737c21/Hep-G2_cbmpy_wgenes_unip_ab.xml'
Hep_G2, errors = validate_sbml_model(Hep_G2_dir+"Hep-G2_cbmpy.xml")
#Hep_G2 = cobra.io.read_sbml_model('./Hep-G2/Hep-G2.xml-5a91f1955e8a9c8a5c5d2ff4a1737c21/Hep-G2.xml')


COBRA errors in validation, check error log for details.
COBRA warnings in validation, check error log for details.


A CBMPY-converted version of the model is loaded, to get the right bounds information.

### Correction du modèle

In [2]:
# Importation du modèle de cellule saine hépatique pour aider à la correction du modèle.
iHep, errors = validate_sbml_model(iHep_dir+"iHepatocytes2322_Cobra.xml")

SBML warnings in validation, check error log for details.
COBRA warnings in validation, check error log for details.


All the functions used to correct the model are gathered in a [separate python file](./Model_correction.py)

In [ ]:
importlib.reload(mc)

<module 'Model_correction' from '/home/jmuller/Documents/Stage_metabo/Model_correction.py'>

---
#### Ajout des gènes associés aux réactions, en les copiant du modèle de cellule saine.
Pour un modèle sans gènes

In [4]:
Hep_G2 = mc.copy_genes(iHep, Hep_G2)


ERROR -- KeyError, The reaction from target model EX_m00097x might not have a counterpart in origin_model.

ERROR -- KeyError, The reaction from target model EX_m00157x might not have a counterpart in origin_model.

ERROR -- KeyError, The reaction from target model EX_m00228x might not have a counterpart in origin_model.

ERROR -- KeyError, The reaction from target model EX_m00242x might not have a counterpart in origin_model.

ERROR -- KeyError, The reaction from target model EX_m00266x might not have a counterpart in origin_model.

ERROR -- KeyError, The reaction from target model EX_m00267x might not have a counterpart in origin_model.

ERROR -- KeyError, The reaction from target model EX_m00268x might not have a counterpart in origin_model.

ERROR -- KeyError, The reaction from target model EX_m00269x might not have a counterpart in origin_model.

ERROR -- KeyError, The reaction from target model EX_m00402x might not have a counterpart in origin_model.

ERROR -- KeyError, The reac

Test to see if the genes have indeed been added to the model :

In [16]:
for reaction in Hep_G2.reactions :
    if "EX_" not in reaction.id :
        print(f"{reaction.id} --- {reaction.bounds}")

HMR_3905 --- (0.0, inf)
HMR_3907 --- (0.0, inf)
HMR_4097 --- (0.0, inf)
HMR_4099 --- (0.0, inf)
HMR_4108 --- (0.0, inf)
HMR_4133 --- (0.0, inf)
HMR_4137 --- (0.0, inf)
HMR_4281 --- (-inf, inf)
HMR_4388 --- (-inf, inf)
HMR_4283 --- (0.0, inf)
HMR_8357 --- (0.0, inf)
HMR_4379 --- (0.0, inf)
HMR_4301 --- (0.0, inf)
HMR_4355 --- (0.0, inf)
HMR_4358 --- (0.0, inf)
HMR_4360 --- (0.0, inf)
HMR_4363 --- (-inf, inf)
HMR_4365 --- (-inf, inf)
HMR_4368 --- (-inf, inf)
HMR_4370 --- (0.0, inf)
HMR_4371 --- (0.0, inf)
HMR_4372 --- (0.0, inf)
HMR_4373 --- (-inf, inf)
HMR_4375 --- (-inf, inf)
HMR_4381 --- (-inf, inf)
HMR_4391 --- (-inf, inf)
HMR_4394 --- (0.0, inf)
HMR_4396 --- (-inf, inf)
HMR_4521 --- (0.0, inf)
HMR_6410 --- (0.0, inf)
HMR_6412 --- (0.0, inf)
HMR_7745 --- (-inf, inf)
HMR_7746 --- (0.0, inf)
HMR_7747 --- (0.0, inf)
HMR_7748 --- (-inf, inf)
HMR_7749 --- (-inf, inf)
HMR_8360 --- (0.0, inf)
HMR_3989 --- (0.0, inf)
HMR_4122 --- (0.0, inf)
HMR_4837 --- (0.0, inf)
HMR_5395 --- (0.0, inf)
HMR

---
#### Modification de la reaction de biomasse pour qu'elle soit optimisable : 
Le métabolite m01602c (cofactors and vitamins) corrompt la réaction, ce métabolite est donc retiré.

In [12]:
Hep_G2.reactions.biomass_components.add_metabolites({"m01602c" : 0.0}, combine=False)

In [14]:
# Addition of the gene annotations :

for gene in Hep_G2.genes :
    if len(gene.id) == 15 :
        gene.annotation["ensembl"] = str(gene.id)

In [ ]:
# Ecriture du modele dans un fichier sbml.

cobra.io.write_sbml_model(Hep_G2, "./models_storage/Hep-G2_temp.xml")

In [37]:
Hep_G2.objective="biomass_components"
Hep_G2.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
m00648x,EX_m00648x,47.37,4,0.31%
m01100x,EX_m01100x,99.47,20,3.29%
m01253x,EX_m01253x,302.1,4,2.00%
m01286x,EX_m01286x,89.63,16,2.37%
m01365x,EX_m01365x,58.96,6,0.59%
m01369x,EX_m01369x,124.4,4,0.82%
m01450x,EX_m01450x,23.85,27,1.06%
m01513x,EX_m01513x,59.71,5,0.49%
m01629x,EX_m01629x,29.48,6,0.29%
m01745x,EX_m01745x,117.6,6,1.17%


--- 
#### Conversion des IDs ENSEMBL en IDs uniprot pour chaque gène.

In [48]:
Hep_G2_unip = mc.convert_ids(Hep_G2)


Creating working copy of the model...

Model copied.

DEBUG : getting uniprot gene id for ENSG00000172955

Gene ensembl id ENSG00000172955 changed to uniprot id : P28332.

DEBUG : getting uniprot gene id for ENSG00000180011

Gene ensembl id ENSG00000180011 changed to uniprot id : Q8N4Q0.

DEBUG : getting uniprot gene id for ENSG00000196344

Gene ensembl id ENSG00000196344 changed to uniprot id : P40394.

DEBUG : getting uniprot gene id for ENSG00000147576

Gene ensembl id ENSG00000147576 changed to uniprot id : B4DFI7.

DEBUG : getting uniprot gene id for ENSG00000187758

Gene ensembl id ENSG00000187758 changed to uniprot id : P07327.

DEBUG : getting uniprot gene id for ENSG00000196616

Gene ensembl id ENSG00000196616 changed to uniprot id : P00325.

DEBUG : getting uniprot gene id for ENSG00000197894

Gene ensembl id ENSG00000197894 changed to uniprot id : D6RAY0.

DEBUG : getting uniprot gene id for ENSG00000248144

Gene ensembl id ENSG00000248144 changed to uniprot id : P00326.

D

#### Cette cellule, si exécutée, va retirer les formules du modèle.
à utiliser par exemple si les formules sont erronées et entraînent des erreurs (ex : présence de parenthèses dans les formules)

In [53]:
Hep_G2_unip_ab = mc.fix_formulas(Hep_G2_unip_ab)

---
#### Enregistrement du modèle corrigé

In [54]:
cobra.io.save_json_model(Hep_G2_unip_ab, Hep_G2_dir+"Hep-G2_json.json")

In [8]:
mc.remove_gene_dupes(Hep_G2_dir+"Hep-G2_json.json", "./Hep-G2_json_nodup.json")

Q92506 already found somewhere else.
P12532 already found somewhere else.
O96024 already found somewhere else.
Q99943 already found somewhere else.
P07327 already found somewhere else.
P00326 already found somewhere else.
P28332 already found somewhere else.
P08319 already found somewhere else.
B4DFI7 already found somewhere else.
P00325 already found somewhere else.
Q8N4Q0 already found somewhere else.
P40394 already found somewhere else.
D6RAY0 already found somewhere else.
P14550 already found somewhere else.
Q9NR19 already found somewhere else.
Q9H6R3 already found somewhere else.
Q9NUB1 already found somewhere else.
O00330 already found somewhere else.
P10515 already found somewhere else.
P11177 already found somewhere else.
P08559 already found somewhere else.
P09622 already found somewhere else.
P29803 already found somewhere else.
Q6ZMR3 already found somewhere else.
Q8IX04 already found somewhere else.
P07195 already found somewhere else.
Q9BYZ2 already found somewhere else.
P

In [2]:
json_temp_model = cobra.io.load_json_model("./Hep-G2_json_nodup.json")

In [4]:
json_temp_model.objective = "artificial_biomass"
json_temp_model

Name,INITModel
Memory address,7f04b197f6a0
Number of metabolites,6705
Number of reactions,5906
Number of genes,2133
Number of groups,0
Objective expression,1.0*artificial_biomass - 1.0*artificial_biomass_reverse_b7cac
Compartments,"Cytosol, Endoplasmic reticulum, Extracellular, Mitochondria, Peroxisome, Lysosome, Golgi apparatus, Nucleus, Boundary"


In [6]:
cobra.io.write_sbml_model(json_temp_model,"./models_storage/Hep-G2_complete.xml")

In [7]:
model_hep_g2= cobra.io.read_sbml_model("models_storage/Hep-G2_complete.xml")

In [8]:
model_hep_g2.solver

In [10]:
model_hep_g2.objective = "artificial_biomass"
model_hep_g2.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
m01308x,EX_m01308x,6.085,0,0.00%
m01365x,EX_m01365x,85.7,0,0.00%
m01369x,EX_m01369x,93.7,0,0.00%
m01393x,EX_m01393x,81.93,0,0.00%
m01513x,EX_m01513x,254.5,0,0.00%
m01629x,EX_m01629x,7.542,0,0.00%
m01668x,EX_m01668x,100,0,0.00%
m01714x,EX_m01714x,250,0,0.00%
m01959x,EX_m01959x,335.1,0,0.00%
m01983x,EX_m01983x,395.3,0,0.00%


In [38]:
for reaction in model_hep_g2.reactions :
    print(reaction.bounds)

(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 1000.0)
(-1000.0, 

## Manips

The goal here is to show that the model is able to simulate a cancer cell's metabolism.

First, we give the model the objective function "artificial biomass" : this is what will be maximized.
Then, before modifying anything, we can look at how the model produces biomass and tweak what isn't right.

In [37]:
# - Chargement du modèle - #

HepG2_v2 = cobra.io.read_sbml_model("./models_storage/Hep-G2_temp.xml")

In [38]:
HepG2_v2.reactions.biomass_components.add_metabolites({"m01602c" : 0.0}, combine=False)

In [48]:
HepG2_v2.solver = "cplex"

In [39]:
HepG2_v2.reactions.biomass_components.bounds

(0.0, inf)

In [122]:
HepG2_v2.reactions.EX_m02630x.bounds = (0.0,0.0) #O2 exchange
HepG2_v2.reactions.EX_m02819x.bounds = (0.0,0.0) #Pyruvate exchange
HepG2_v2.reactions.EX_m01965x.bounds = (-1000,1000.0) #Glucose exchange
HepG2_v2.reactions.EX_m01286x.bounds = (0.0,0.0) #ADP-Glucose exchange
HepG2_v2.reactions.EX_m01840x.bounds = (0.0,0.0) #Fructose exchange
HepG2_v2.reactions.EX_m01743x.bounds = (-1000.0,1000.0) #D-Ribulose exchange
HepG2_v2.reactions.EX_m02453x.bounds = (0.0,0.0) #Mannose exchange
HepG2_v2.reactions.EX_m02470x.bounds = (0.0,0.0) #Methanol exchange

# --> After Methanol cut, Glucose uptake flux went from 0.0 to -190.0
# Without O2, glucose uptake went down to -23

In [123]:
HepG2_v2.objective = "biomass_components"
sol = HepG2_v2.optimize()
sol

,fluxes,reduced_costs
EX_m00097x,-0.000000,0.0
EX_m00157x,939.369842,0.0
EX_m00228x,0.000000,0.0
EX_m00242x,0.000000,0.0
EX_m00266x,-0.000000,0.0
...,...,...
HMR_9715,0.000000,0.0
HMR_9730,0.000000,0.0
HMR_9736,-141.476597,0.0
biomass_components,26.421410,0.0


Changing all reaction bounds from inf to 1000 or -inf to -1000 :

In [40]:
for reaction in HepG2_v2.reactions :
    ub = 1000.0 if reaction.upper_bound == float("inf") or reaction.upper_bound ==1000.0 else 0.0
    lb = -1000.0 if reaction.lower_bound == float("-inf") or reaction.lower_bound == -1000.0 else 0.0
    reaction.bounds = (lb,ub)
    print(f"DEBUG --- reaction {reaction.id}\noriginal bounds : ({reaction.lower_bound},{reaction.upper_bound})\nnew bounds : ({lb},{ub})\n\t########\n")

DEBUG --- reaction EX_m00097x
original bounds : (-1000.0,1000.0)
new bounds : (-1000.0,1000.0)
	########

DEBUG --- reaction EX_m00157x
original bounds : (-1000.0,1000.0)
new bounds : (-1000.0,1000.0)
	########

DEBUG --- reaction EX_m00228x
original bounds : (-1000.0,1000.0)
new bounds : (-1000.0,1000.0)
	########

DEBUG --- reaction EX_m00242x
original bounds : (-1000.0,1000.0)
new bounds : (-1000.0,1000.0)
	########

DEBUG --- reaction EX_m00266x
original bounds : (-1000.0,1000.0)
new bounds : (-1000.0,1000.0)
	########

DEBUG --- reaction EX_m00267x
original bounds : (-1000.0,1000.0)
new bounds : (-1000.0,1000.0)
	########

DEBUG --- reaction EX_m00268x
original bounds : (-1000.0,1000.0)
new bounds : (-1000.0,1000.0)
	########

DEBUG --- reaction EX_m00269x
original bounds : (-1000.0,1000.0)
new bounds : (-1000.0,1000.0)
	########

DEBUG --- reaction EX_m00402x
original bounds : (-1000.0,1000.0)
new bounds : (-1000.0,1000.0)
	########

DEBUG --- reaction EX_m00545x
original bounds 

#### Flux des réactions d'échanges

In [124]:
for reaction in HepG2_v2.reactions :
    if reaction.flux != 2000000 :
        if "EX_" in reaction.id :

            # Jolification 
            spaces = 12
            spaces_str = ""
            for i in str(round(reaction.flux)) :
                spaces -= 1
            if "EX_t" in reaction.id :
                spaces -= 1
            for j in range(spaces) :
                spaces_str += " "
            # Fin de la jolification

            m = [metab for metab in reaction.metabolites][0]    

            print(f"{reaction.id} : {round(reaction.flux)}{spaces_str}ub : {reaction.upper_bound}\t---\t\tmetabolites : \t id : {m.id} --- metabolite name : {m.name}")

EX_m00097x : 0           ub : 1000.0	---		metabolites : 	 id : m00097x --- metabolite name : (5-L-glutamyl)-L-amino acid
EX_m00157x : 939         ub : 1000.0	---		metabolites : 	 id : m00157x --- metabolite name : (R)-3-hydroxybutanoate
EX_m00228x : 0           ub : 1000.0	---		metabolites : 	 id : m00228x --- metabolite name : 1,1-dichloroethylene
EX_m00242x : 0           ub : 1000.0	---		metabolites : 	 id : m00242x --- metabolite name : 1,2-dibromoethane
EX_m00266x : 0           ub : 1000.0	---		metabolites : 	 id : m00266x --- metabolite name : 10-formyl-THF
EX_m00267x : 0           ub : 1000.0	---		metabolites : 	 id : m00267x --- metabolite name : 10-formyl-THF-glu(5)
EX_m00268x : 0           ub : 1000.0	---		metabolites : 	 id : m00268x --- metabolite name : 10-formyl-THF-glu(6)
EX_m00269x : 0           ub : 1000.0	---		metabolites : 	 id : m00269x --- metabolite name : 10-formyl-THF-glu(7)
EX_m00402x : 0           ub : 1000.0	---		metabolites : 	 id : m00402x --- metabolite nam

#### Definitinon of the print_reactions function, used, when needed, to print reactions in a beautiful way
This function can print (v=True) or return (v=False) two strings reprensenting the reaction.
One identifies the reactants and products by their name, the other by their ID in the model.
The arrow indicating the reaction direction is determined from the flux variable.

In [ ]:
def print_reactions(reaction, flux = 0.0, v=True):
    '''
    Affiche les reactions d'un modele de façon plus lisible
    '''
    liste_reactif = [i for i in reaction.reactants]
    liste_produit = [i for i in reaction.products]
    string = ""
    string2 = ""
    if flux >= float('0'):
            fleche = "-->"
    elif flux <= float('0') :
            fleche = "<--"
    else:
            fleche = "<=>"
    
    for i in liste_reactif:
        if i != liste_reactif[-1]:
            string += str(float(abs(reaction.metabolites[i])))
            string += " "
            string += str(i.name)
            string += " + "
            
            string2 += str(float(abs(reaction.metabolites[i])))
            string2 += " "
            string2 += str(i.id)
            string2 += " + "
            
        else:
            string += str(float(abs(reaction.metabolites[i])))
            string += " "
            string += str(i.name)
            string += " "
            
            string2 += str(float(abs(reaction.metabolites[i])))
            string2 += " "
            string2 += str(i.id)
            string2 += " "
            
    string += fleche
    string += " "
    
    string2 += fleche
    string2 += " "
    
    
    for i in liste_produit:
        if i != liste_produit[-1]:
            string += str(float(abs(reaction.metabolites[i])))
            string += " "
            string += str(i.name)
            string += " + "
            
            string2 += str(float(abs(reaction.metabolites[i])))
            string2 += " "
            string2 += str(i.id)
            string2 += " + "
            
        else:
            string += str(float(abs(reaction.metabolites[i])))
            string += " "
            string += str(i.name)
            
            string2 += str(float(abs(reaction.metabolites[i])))
            string2 += " "
            string2 += str(i.id)
    if v :    
        print(string)
        print("")
        print(string2)
    else :
        return(string, string2)

### Comparison of metabolites in the biomass reaction for the healthy and cancerous models

In [21]:
abm = []
for metabolite in Hep_G2.reactions.artificial_biomass.metabolites :
    abm.append(metabolite.id)

In [22]:
bm = []
for metabolite in Hep_G2.reactions.biomass_components.metabolites:
    bm.append(metabolite.id)

In [23]:
iHep_bm = []
for metabolite in iHep.reactions.biomass_components.metabolites :
    iHep_bm.append(metabolite.id)

Metabolites that are in the **artificial_biomass** reaction but not in **iHep's biomass reaction**

In [24]:
for m in abm :
    if m not in iHep_bm :
        print(m, Hep_G2.metabolites.get_by_id(m).name)

temp001x biomass


Metabolites that are in **iHep's biomass reaction** but not in the **artificial_biomass** reaction

In [25]:
for m in iHep_bm :
    if m not in abm :
        print(m, Hep_G2.metabolites.get_by_id(m).name)

m01602c cofactors and vitamins


Metabolites that are in **Hep_G2's original biomass reaction** but not in the **artificial_biomass reaction**

In [26]:
for m in bm : 
    if m not in abm : 
        print(m, Hep_G2.metabolites.get_by_id(m).name)

m01602c cofactors and vitamins


Metabolites that are in the **artificial_biomass reaction** but not in **Hep_G2's original biomass reaction**

In [27]:
for m in abm : 
    if m not in bm : 
        print(m,Hep_G2.metabolites.get_by_id(m).name)

#### Cell used to check reaction bounds

In [9]:
for reaction in Hep_G2.reactions :
    if reaction.bounds != (float("-inf"), float("inf")) and "EX_" not in reaction.id :
        print(f"{reaction.id}, {reaction.bounds}")

HMR_3905, (0.0, inf)
HMR_3907, (0.0, inf)
HMR_4097, (0.0, inf)
HMR_4099, (0.0, inf)
HMR_4108, (0.0, inf)
HMR_4133, (0.0, inf)
HMR_4137, (0.0, inf)
HMR_4283, (0.0, inf)
HMR_8357, (0.0, inf)
HMR_4379, (0.0, inf)
HMR_4301, (0.0, inf)
HMR_4355, (0.0, inf)
HMR_4358, (0.0, inf)
HMR_4360, (0.0, inf)
HMR_4370, (0.0, inf)
HMR_4371, (0.0, inf)
HMR_4372, (0.0, inf)
HMR_4394, (0.0, inf)
HMR_4521, (0.0, inf)
HMR_6410, (0.0, inf)
HMR_6412, (0.0, inf)
HMR_7746, (0.0, inf)
HMR_7747, (0.0, inf)
HMR_8360, (0.0, inf)
HMR_3989, (0.0, inf)
HMR_4122, (0.0, inf)
HMR_4837, (0.0, inf)
HMR_5395, (0.0, inf)
HMR_5396, (0.0, inf)
HMR_9727, (0.0, inf)
HMR_5397, (0.0, inf)
HMR_5398, (0.0, inf)
HMR_5399, (0.0, inf)
HMR_5400, (0.0, inf)
HMR_5401, (0.0, inf)
HMR_8568, (0.0, inf)
HMR_8570, (0.0, inf)
HMR_8572, (0.0, inf)
HMR_8574, (0.0, inf)
HMR_8576, (0.0, inf)
HMR_8578, (0.0, inf)
HMR_8591, (0.0, inf)
HMR_8592, (0.0, inf)
HMR_8587, (0.0, inf)
HMR_8589, (0.0, inf)
HMR_3944, (0.0, inf)
HMR_4130, (0.0, inf)
HMR_4132, (0.

### Réactions d'échange

In [54]:
for reaction in Hep_G2_unip_ab.reactions :
    if "EX_" in reaction.id :
        if reaction.id != "EX_m01286x" and reaction.id != "EX_m01965x" and reaction.id != "EX_m01252x":
            reaction.bounds = (0.0,0.0)
            #print_reactions(reaction)
            #print("")
            #print(f"Flux : {reaction.flux}\n")
            #print(f"{reaction.bounds} --- {reaction.id}")
            #print("\n\n")
        else :
            reaction.bounds = (float("-inf"), float("inf"))

In [64]:
print(Hep_G2_unip_ab.reactions.EX_temp001x)

EX_temp001x: temp001x --> 


In [61]:
for reaction in Hep_G2_unip_ab.reactions :
    if "EX_" in reaction.id :
        metab = [m.id for m in reaction.metabolites]
        if "m01965x" in metab or "m01286x" in metab :
            
            reaction.bounds = (float('-inf'),0.0)
            print(reaction, reaction.bounds)

EX_m01286x: m01286x <--  (-inf, 0.0)
EX_m01965x: m01965x <--  (-inf, 0.0)


In [28]:
Hep_G2.reactions.artificial_biomass

Reaction identifier,artificial_biomass
Name,
Memory address,0x7fac785d18b0
Stoichiometry,m01307c + m01365c + m01369c + m01370c + m01450c + m01451r + m01589c + m01628c + m01721n + m01722n + m01974c + m01975c + m01986c + m02125c + m02184c + m02360c + m02392c + m02426c + m02471c + m02724c... alanine + arginine + asparagine + aspartate + cholesterol + cholesterol-ester pool + CL pool + cysteine + DNA + DNA-5-methylcytosine + glutamate + glutamine + glycine + histidine + isoleucine +...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [31]:
for biomass_metabolite in Hep_G2.reactions.artificial_biomass.metabolites :
    print(biomass_metabolite.name,'---', biomass_metabolite.id)

biomass --- temp001x
glutamine --- m01975c
RNA --- m02847c
tryptophan --- m03089c
threonine --- m02993c
proline --- m02770c
lysine --- m02426c
arginine --- m01365c
serine --- m02896c
tyrosine --- m03101c
isoleucine --- m02184c
asparagine --- m01369c
cysteine --- m01628c
methionine --- m02471c
glycine --- m01986c
aspartate --- m01370c
glycogen --- m03161c
phenylalanine --- m02724c
alanine --- m01307c
histidine --- m02125c
valine --- m03135c
leucine --- m02360c
glutamate --- m01974c
DNA --- m01721n
DNA-5-methylcytosine --- m01722n
cholesterol --- m01450c
CL pool --- m01589c
cholesterol-ester pool --- m01451r
lipid droplet --- m02392c
PI pool --- m02750c
SM pool --- m02908c
phosphatidate-LD-TAG pool --- m02733c


### Vérification effet Warburg

In [48]:
# Recherche des réactions impliquant l'O2
O2_metab = []
print("O2 :")
for metab in Hep_G2.metabolites :
    if 'O2' in metab.name.split('_')[0][:2] and len(metab.name.split('_')[0]) < 6 :
        O2_metab.append(metab)       
        print(f'{metab.name} --- {metab.id} --- {metab.compartment}')
print("\nReactions associées à O2\n")
for O2 in O2_metab :
    for reac in O2.reactions :
        if reac.flux != 0.0 and len(reac.compartments) >1:
            print(f"{reac.id} --- {reac.flux}\n")
            print_reactions(reac)
            print("\n####====================####\n")

O2 :
O2 --- m02630c --- C_c
O2- --- m02631c --- C_c
O2 --- m02630m --- C_m
O2- --- m02631m --- C_m
O2 --- m02630n --- C_n
O2- --- m02631n --- C_n
O2 --- m02630p --- C_p
O2- --- m02631p --- C_p
O2 --- m02630r --- C_r
O2 --- m02630s --- C_s
O2- --- m02631s --- C_s
O2 --- m02630x --- C_x
O2- --- m02631x --- C_x

Reactions associées à O2

HMR_4896 --- 236.39280919611895

1.0 O2 <=> 1.0 O2

1.0 m02630c <=> 1.0 m02630s

####====================####

HMR_8414 --- -3.744976978240768e-15

1.0 O2- <=> 1.0 O2-

1.0 m02631c <=> 1.0 m02631p

####====================####

HMR_6914 --- -37.0677259254506

4.0 ferrocytochrome C + 8.0 H+ + 1.0 O2 <=> 4.0 ferricytochrome C + 4.0 H+ + 2.0 H2O

4.0 m01826m + 8.0 m02039m + 1.0 m02630m <=> 4.0 m01824m + 4.0 m02039c + 2.0 m02040m

####====================####

HMR_8414 --- -3.744976978240768e-15

1.0 O2- <=> 1.0 O2-

1.0 m02631c <=> 1.0 m02631p

####====================####

HMR_9048 --- -232.75219564567547

1.0 O2 <=> 1.0 O2

1.0 m02630x <=> 1.0 m02630s

###

In [34]:
Hep_G2.reactions.HMR_4280.bounds = (0.0,0.0)

In [30]:
for reaction in Hep_G2.metabolites.get_by_id('m02819c').reactions :
    print_reactions(reaction)
    print(reaction.id)

1.0 PEP + 1.0 UDP --> 1.0 pyruvate + 1.0 UTP

1.0 m02696c + 1.0 m03106c --> 1.0 m02819c + 1.0 m03130c
HMR_4210
1.0 (R)-3-hydroxybutanoate + 1.0 pyruvate --> 1.0 (R)-3-hydroxybutanoate + 1.0 pyruvate

1.0 m00157c + 1.0 m02819s --> 1.0 m00157s + 1.0 m02819c
HMR_6021
1.0 malate + 1.0 NADP+ --> 1.0 CO2 + 1.0 H+ + 1.0 NADPH + 1.0 pyruvate

1.0 m02439c + 1.0 m02554c --> 1.0 m01596c + 1.0 m02039c + 1.0 m02555c + 1.0 m02819c
HMR_4089
1.0 serine --> 1.0 NH3 + 1.0 pyruvate

1.0 m02896c --> 1.0 m02578c + 1.0 m02819c
HMR_3883
1.0 L-lactate + 1.0 pyruvate --> 1.0 L-lactate + 1.0 pyruvate

1.0 m02403c + 1.0 m02819s --> 1.0 m02403s + 1.0 m02819c
HMR_6018
1.0 H2O + 1.0 salsolinol-1-carboxylate --> 1.0 dopamine + 1.0 pyruvate

1.0 m02040c + 1.0 m02876c --> 1.0 m01736c + 1.0 m02819c
HMR_6738
1.0 H2O + 1.0 se-methyl-L-selenocysteine --> 1.0 methaneselenol + 1.0 NH3 + 1.0 pyruvate

1.0 m02040c + 1.0 m02895c --> 1.0 m02469c + 1.0 m02578c + 1.0 m02819c
HMR_7141
1.0 dGDP + 1.0 PEP --> 1.0 dGTP + 1.0 pyruvate

In [48]:
# Recherche des réactions impliquant le lactate
Lactate_metab = []
print("GLUCOSE :")
for metab in Hep_G2_unip_ab.metabolites :
    if 'glucose' in metab.name :
        Lactate_metab.append(metab)       
        print(f'{metab.name} --- {metab.id} --- {metab.compartment}')

GLUCOSE :
4,6-dideoxy-4-oxo-dTDP-D-glucose --- m00946c --- C_c
ADP-glucose --- m01286c --- C_c
ADP-glucose --- m01286s --- C_s
ADP-glucose --- m01286x --- C_x
beta-D-glucose --- m01388c --- C_c
beta-D-glucose-6-phosphate --- m01389c --- C_c
dTDP-glucose --- m01751c --- C_c
glucose --- m01965c --- C_c
glucose --- m01965g --- C_g
glucose --- m01965l --- C_l
glucose --- m01965r --- C_r
glucose --- m01965s --- C_s
glucose --- m01965x --- C_x
glucose-1-phosphate --- m01967c --- C_c
glucose-6-phosphate --- m01968c --- C_c
glucose-6-phosphate --- m01968r --- C_r
UDP-glucose --- m03108c --- C_c
UDP-glucose --- m03108r --- C_r


In [49]:
# Recherche des réactions impliquant le lactate
Lactate_metab = []
print("LACTATES :")
for metab in Hep_G2_unip_ab.metabolites :
    if 'L-lactate' in metab.name :
        Lactate_metab.append(metab)       
        print(f'{metab.name} --- {metab.id} --- {metab.compartment}')
print("\nReactions associées aux lactates\n")
for lac in Lactate_metab :
    for reac in lac.reactions :
        if reac.flux != 0.0 :
            print(f"ID : {reac.id} --- FLUX : {reac.flux}\n")
            print_reactions(reac, reac.flux)
            print("\n####====================####\n")

LACTATES :
L-lactate --- m02403c --- C_c
L-lactate --- m02403m --- C_m
L-lactate --- m02403p --- C_p
L-lactate --- m02403s --- C_s
L-lactate --- m02403x --- C_x

Reactions associées aux lactates



In [79]:
metab_reactions_to_file(Hep_G2, "L-Lactate", "lac_flux.txt")

L-Lactate : 

Réactions associées au L-Lactate
 :


In [78]:
def metab_reactions_to_file(model, metab_name, outfile) :
    metab_list = []
    
    string = f"{metab_name} : \n"
    
    for metab in model.metabolites :
        if metab_name in metab.name :
            metab_list.append(metab)
            string += metab.name + " --- " + {metab.id} + " --- " + {metab.compartment} + "\n"
            print(string)
    string += "\nRéactions associées au " + metab_name + "\n :"
    
    for m in metab_list :
        for reac in m.reactions :
            if reac.flux != 0.0 :
                s1 = print_reactions(reac, reac.flux, False)[0]
                s2 = print_reactions(reac, reac.flux, False)[1]
                string += "ID : " + reac.id + " --- FLUX : " + reac.flux +"\n"
                string += s1 + "\n" + s2 + "\n"
                string += "\n####====================####\n"
    
    with open(outfile, 'w') as out :
        out.write(string)
    print(string)

### Model from 2012 INIT

In [53]:
init_2012 = cobra.io.read_sbml_model('./2012INIT_liver/iLiverCancer1788.xml')

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
urn:miriam:9606 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
urn:miriam:urn:miriam:obo.chebi:CHEBI:17499 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
urn:miriam:urn:miriam:obo.chebi:CHEBI:17499 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
urn:miriam:urn:miriam:obo.chebi:CHEBI:17499 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
urn:miriam:urn:miriam:obo.chebi:CHEBI:17513 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
urn:miriam:urn:miriam:obo.chebi:CHEBI:17908 does not conform to 'http(s)://identif

urn:miriam:urn:miriam:obo.chebi:CHEBI:16946 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m66 "L-Kynurenine">
urn:miriam:urn:miriam:obo.chebi:CHEBI:28683 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m67 "Kynurenine">
urn:miriam:urn:miriam:obo.chebi:CHEBI:1547 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m68 "3-Hydroxykynurenine">
urn:miriam:urn:miriam:obo.chebi:CHEBI:17380 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, 

urn:miriam:urn:miriam:obo.chebi:CHEBI:17172 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m98 "Deoxyguanosine">
urn:miriam:urn:miriam:obo.chebi:CHEBI:17172 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m99 "Deoxyguanosine">
urn:miriam:urn:miriam:obo.chebi:CHEBI:17172 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m100 "Deoxyguanosine">
urn:miriam:urn:miriam:obo.chebi:CHEBI:17172 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discourage

Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m129 "dGMP">
urn:miriam:urn:miriam:obo.chebi:CHEBI:17345 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m130 "GMP">
urn:miriam:urn:miriam:obo.chebi:CHEBI:17345 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m131 "GMP">
urn:miriam:urn:miriam:obo.chebi:CHEBI:17345 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m132 "GMP">
urn:miriam:urn:miriam:obo.chebi:CHEBI:17345 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://i

urn:miriam:urn:miriam:obo.chebi:CHEBI:17552 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m161 "GDP">
urn:miriam:urn:miriam:obo.chebi:CHEBI:17552 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m162 "GDP">
urn:miriam:urn:miriam:obo.chebi:CHEBI:17552 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m163 "GDP">
urn:miriam:urn:miriam:obo.chebi:CHEBI:17552 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula inst

...

Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_8123 "Gamma-glutamyltransferase 5 light chain">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_8123 "Gamma-glutamyltransferase 5 light chain">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_8124 "Gamma-glutamyltransferase 5 light chain">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_8124 "Gamma-glutamyltransferase 5 light chain">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_8125 "Gamma-glutamyltransferase 5 light chain">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_8125 "Gamma-glutamyltransferase 5 light chain">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_8126 "Gamma-glutamyltransferase 5 light chain">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_8126 "Gamma-glutamyltransferase 5 light chain">'
Missing lower flux bound set to '-1000.0' for reaction: '<Re

Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_8494 "SIMILAR TO SEPIAPTERIN REDUCTASE">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_8494 "SIMILAR TO SEPIAPTERIN REDUCTASE">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_8499 "4-trimethylaminobutyraldehyde dehydrogenase">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_8499 "4-trimethylaminobutyraldehyde dehydrogenase">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_8525 "Gamma-glutamylcyclotransferase">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_8525 "Gamma-glutamylcyclotransferase">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_8530 "NADPH oxidase 1">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_8530 "NADPH oxidase 1">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_8531 "NADPH oxidase 1">'
Missing upper flux bound set to '1000.

Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_8871 "Transient receptor potential cation channel subfamily M member 2">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_8878 "Pyridoxal phosphate phosphatase PHOSPHO2">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_8878 "Pyridoxal phosphate phosphatase PHOSPHO2">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_8889 "Guanylate kinase">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_8889 "Guanylate kinase">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_8893 "Nicotinate phosphoribosyltransferase">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_8893 "Nicotinate phosphoribosyltransferase">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_8894 "Probable gluconokinase">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_8894 "Probable gluconokinase">'
Mi

Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_9279 "Amine oxidase [flavin-containing] B">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_9284 "Catechol O-methyltransferase">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_9284 "Catechol O-methyltransferase">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_9285 "Catechol O-methyltransferase">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_9285 "Catechol O-methyltransferase">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_9286 "Catechol O-methyltransferase">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_9286 "Catechol O-methyltransferase">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_9287 "Catechol O-methyltransferase">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_9287 "Catechol O-methyltransferase">'
Missing lower flux bound set to '-1

Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_9488 "4-hydroxyphenylpyruvate dioxygenase">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_9488 "4-hydroxyphenylpyruvate dioxygenase">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_9491 "Cytochrome P450 3A4">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_9491 "Cytochrome P450 3A4">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_9499 "phosphatidylinositol-3,4-bisphosphate 4-phosphatase">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_9499 "phosphatidylinositol-3,4-bisphosphate 4-phosphatase">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_9533 "Leukotriene-B(4) omega-hydroxylase 1">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction R_9533 "Leukotriene-B(4) omega-hydroxylase 1">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction R_9534 "Aldo-keto reduc

In [62]:
init_2012

Name,iLiverCancer1788
Memory address,7fac77670070
Number of metabolites,4601
Number of reactions,2794
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,"cytosol, endoplasmic reticulum, mitochondria, extracellular, nucleus, peroxisomes, Golgi apparatus, lysosomes"


## iHepatocytes model : Modèle de cellule du foie non cancéreuse, utilisé dans l'étude des perturbations métaboliques causées par NAFLD

[Article](./iHepatocytes2322/iHepatocytes.pdf)

In [60]:
# Model used to study NAFLD, not cancer

iHep, errors = validate_sbml_model('./iHepatocytes2322/iHepatocytes2322_Cobra.xml')

KeyboardInterrupt: 

In [89]:
for reaction in iHep.reactions :
    genes = [gene.id for gene in reaction.genes]
    for g in genes :
        UniprotkbClient.fetch_one(g)
        
    #print(genes)

HTTPError: 400 Client Error: Bad Request for url: https://rest.uniprot.org/uniprotkb/ENSG00000180011.json

In [16]:
iHep

Name,iHepatocytes2322
Memory address,7efbcb106c70
Number of metabolites,5242
Number of reactions,7930
Number of genes,2322
Number of groups,135
Objective expression,0
Compartments,"c, s, l, r, m, p, g, n"


In [ ]:
iHep.objective = 'biomass_components'
iHep.summary()

HMR2 = cobra.io.read_sbml_model('hmr/HMRdatabase2_00_Cobra.xml-b36c4d8bbafd1a3b8bf3b49e8b7cad35/HMRdatabase2_00_Cobra.xml')

HMR2.objective = "biomass_components"
HMR2.summary()

#### Reaction bounds correction for Hep_G2 model :
For the case where bounds are 'inf' or '-inf'

In [ ]:
for i in Hep_G2.reactions:
	if i.upper_bound == float('inf'):
		i.upper_bound = float(1000)

	if i.lower_bound == float('-inf'):
		i.lower_bound = float(-1000)

#### Checking that the bounds of the shared reactions between Hep_G2 and iHep are the same

If they are different, the cell will print each reaction that has different bounds and its bounds.
The cell will also copy-paste the bounds from the iHep model's reactions to Hep_G2's

In [ ]:
i = 0
for reaction_Hep_G2 in Hep_G2.reactions :
    if reaction_Hep_G2 in iHep.reactions :
        #Comparaison des bounds pour une même réaction entre les deux modèles :
        iHep_reaction_correspondance = iHep.reactions.get_by_id(reaction_Hep_G2.id)
        if reaction_Hep_G2.bounds != iHep_reaction_correspondance.bounds :
            print(f"bounds in Hep_G2 : {reaction_Hep_G2.bounds}\n---\nbounds in iHep : {iHep_reaction_correspondance.bounds}\n#########\n")
            Hep_G2.reactions.get_by_id(reaction_Hep_G2.id).bounds = iHep_reaction_correspondance.bounds
        else :
            #print(f"Reaction {reaction_Hep_G2} (b : {reaction_Hep_G2.bounds}) and {iHep_reaction_correspondance} (b: {iHep_reaction_correspondance.bounds}) have the same bounds\n\n")
            i+=1

### Comparaison des réactions en fonction de leur ID et vérification des métabolites composant cette réaction.

#### Comparaison des réactions du model1 avec celles du model2. Les Réactions seront cherchées dans le model2 à partir de leurs id du model1.

In [ ]:
def compare_models_reactions(model1, model2, v = False, fig=True) :
    in_both = 0
    only_in_model1 = 0
    only_in_model2 = 0
    model2_reactions = [id_model2.id for id_model2 in model2.reactions]
    model1_reactions = [id_model1.id for id_model1 in model1.reactions]
    
    only_in_model1_list = []
    only_in_model2_list = []
    in_both_list = []
    for rea in model1.reactions :
        
        if rea.id in model2_reactions :
            model2_id = model2.reactions.get_by_id(rea.id)
            in_both +=1
            in_both_list.append(rea)
        else :
            model2_id = "False"
            only_in_model1+=1
            only_in_model1_list.append(rea)
        if v :
            print(f"reaction_iHep : {rea}\n {rea.name}\n---\nreaction_init: {model2.id} \n\n=====\n\n")
        
    for rea in model2.reactions :
        if rea.id in model1_reactions :
            pass
        else :
            only_in_model2 +=1
            only_in_model2_list.append(rea)
    if fig :
        fig, ax = plt.subplots()
        both_fig = ax.bar("In model1 and model2", in_both)
        only_1_fig = ax.bar("only in model1", only_in_model1)
        only_2_fig = ax.bar("only in model2", only_in_model2)

        fig.legend([both_fig, only_1_fig, only_2_fig], [f"{model1.name} vs {model2.name}", model1.name, model2.name])
        return fig, ax
    else :
        return only_in_model1_list, only_in_model2_list, in_both_list

In [ ]:
fig, ax = compare_models_reactions(Hep_G2, iHep)

In [ ]:
l1, l2, _ = compare_models_reactions(Hep_G2, iHep, fig=False)
print(len(l1))
for reaction_to_print in l1 : 
    print(reaction_to_print.id)

In [ ]:
biomass_Hep_G2 = Hep_G2.reactions.biomass_components


print_reactions(biomass_Hep_G2)

In [ ]:
Hep_G2.add_reactions([biomass_iHep])

In [ ]:
biomass_Hep_G2 = Hep_G2.reactions.biomass_components.metabolites

In [ ]:
temp001x= cobra.Metabolite(
    'temp001x',
    formula='',
    name='biomass_',
    compartment='C_c')
output_biom = cobra.Reaction('EX_temp001x')
output_biom.lower_bound = 0.0
output_biom.upper_bound = 1000.0
output_biom.add_metabolites({temp001x :-1.0})
Hep_G2.add_reactions([output_biom])

In [ ]:
Hep_G2.reactions.EX_temp001x.lower_bound = 0.0

In [ ]:
Hep_G2.reactions.EX_temp001x

In [ ]:
biomass_iHep = iHep.reactions.biomass_components
biomass_iHep.metabolites

In [ ]:
biomass_components_ids = []
biomass_components_formulas = []
for metabolite in model_G2.reactions.get_by_id('biomass_components').metabolites :
    print(metabolite, metabolite.name)
    biomass_components_ids.append(metabolite.id)
    #biomass_components_formulas.append(metabolite.name.split('_')[1])

In [ ]:
# Checking which reactions are exchange reactions :

for reaction in model_iH.reactions :
    if len(reaction.compartments) >1 :
        print_reactions(reaction)
        
        #print(reaction, reaction.compartments, [m.name for m in reaction.metabolites])
        #print("")

In [ ]:
biomass_components_

In [ ]:
for metab in model_init.metabolites :
    print(metab.name)

In [ ]:
for m_id in biomass_components_ids :
    print(model_init.reactions.get_by_id(m_id).name)

In [ ]:
model_iH.objective = "biomass_components"
model_iH.summary()
